# Basic usage of RNN embeddings

### You can feed multiple indices to the embedding layer.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

word_to_ix = {'hello': 0, 'world': 1}
# lookup multiple indices at a time
lookup_tensor = torch.tensor([word_to_ix['hello'], 1], dtype=torch.long)
embed = nn.Embedding(2, 5)
result = embed(lookup_tensor)
print(result)

tensor([[ 0.4389,  0.1606,  0.6134,  0.9061,  3.2656],
        [ 0.9126,  1.8042,  0.1382, -0.8343, -1.1398]],
       grad_fn=<EmbeddingBackward>)


In [8]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


train_grams = [[(test_sentence[i], test_sentence[i+1]), test_sentence[i+2]]
    for i in range(len(test_sentence) - 2)]
print(train_grams[:2])

vocab = set(test_sentence)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class RNN(nn.Module):
    def __init__(self, context_size, dim, vocab_size):
        super(RNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, dim)
        self.linear1 = nn.Linear(context_size * dim, 128) # 128 is hidden size
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, data):
        out = self.embed(data).view((1, -1))
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.log_softmax(out, dim=1)
        return out
        
model = RNN(CONTEXT_SIZE, EMBEDDING_DIM, vocab_size)
criterion = nn.NLLLoss()
optimimzer = optim.Adam(model.parameters(), lr=0.1)


for epoch in range(10):
    total_loss = 0.0
    for words, target in train_grams:        
        # reset to prevent accumulation
        model.zero_grad()
        
        input_tensor = torch.tensor([word_to_ix[w] for w in words], dtype=torch.long)
        target_tensor = torch.tensor([word_to_ix[target]], dtype=torch.long)
                
        output = model(input_tensor)
        loss = criterion(output, target_tensor)
        loss.backward()
        optimimzer.step()
        total_loss += loss
        
    print("total_loss: {}".format(total_loss))

[[('When', 'forty'), 'winters'], [('forty', 'winters'), 'shall']]
total_loss: 1773.1373291015625
total_loss: 691.1527709960938
total_loss: 681.7239990234375
total_loss: 621.2034301757812
total_loss: 580.2962646484375
total_loss: 535.4334716796875
total_loss: 627.6587524414062
total_loss: 538.5535278320312
total_loss: 539.4458618164062
total_loss: 532.6720581054688


### 不自己闭眼实现一遍，根本意识不到很多小的细节，如果不注意，程序就会千疮百孔。
- `super(RNN, self).__init__`
- `nn.Embedding()` the first dimension is the total vocab size
- list comprehension for dictionary：`{word: i for i, word in enumerate(vocab)}`
- `view((1, -1))`, should be double brackets just like `reshape` in numpy
- apply `relu` after first FC layer
- manually run `soft_max` at the very end since we're using `NLLLoss()`
- initialize optimizer `optim.Adam` or `optim.SGD()`
- input tensor and output tensor are just plain indices wrapped in a tensor
- forgot to run `loss.backward()`